In [5]:
!pip install -r requirements.txt

   ---------------------------------------- 0.0/103.5 kB ? eta -:--:--
   --------------------------- ------------ 71.7/103.5 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 103.5/103.5 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 22.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/205.2 kB ? eta -:--:--
   --------------------------------------- 205.2/205.2 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: groq
    Found existing installation: groq 1.0.0
    Uninstalling groq-1.0.0:
      Successfully uninstalled groq-1.0.0


In [7]:
from groq import Groq
from dotenv import load_dotenv
import os

In [8]:
load_dotenv()
groq_api_key = os.getenv('GROQ_API_KEY')

In [9]:
client = Groq()

In [12]:
completion = client.chat.completions.create(
    model="openai/gpt-oss-120b",
    messages=[
      {
        "role": "user",
        "content": ""
      }
    ],
    temperature=1,
    top_p=1,
    stream=True,
    stop=None
)

for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")


Hello! How can I assist you today?

# LangChain Agents Examples

LangChain agents allow AI models to interact with tools and take actions based on model decisions. Agents can decide which tools to use and in what sequence to accomplish a task.

In [ ]:
# Install LangChain dependencies
!pip install langchain langchain-community langchain-core python-dotenv -q

## Example 1: Simple Tool Definition and Agent Setup

Basic example showing how to define tools and create an agent that can use them.

In [ ]:
from langchain.tools import tool
from langchain_core.tools import Tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub
from langchain_groq import ChatGroq

# Define some simple tools
@tool
def calculate_calories(meal: str, portion_size: str) -> str:
    """Calculate calories for a given meal and portion size."""
    # Simplified example
    meal_calories = {
        "chicken breast": 165,
        "rice": 206,
        "broccoli": 34,
        "salmon": 280,
        "apple": 95
    }
    calories = meal_calories.get(meal.lower(), 0)
    return f"{meal} ({portion_size}) contains approximately {calories} calories per 100g"

@tool
def get_nutritional_info(food_item: str) -> str:
    """Get nutritional information for a food item including protein, carbs, and fat."""
    nutrition_db = {
        "chicken breast": "Protein: 31g, Carbs: 0g, Fat: 3.6g (per 100g)",
        "rice": "Protein: 2.7g, Carbs: 28g, Fat: 0.3g (per 100g)",
        "broccoli": "Protein: 2.8g, Carbs: 7g, Fat: 0.4g (per 100g)",
        "salmon": "Protein: 25g, Carbs: 0g, Fat: 13g (per 100g)"
    }
    return nutrition_db.get(food_item.lower(), "Food item not found in database")

# Create the agent
llm = ChatGroq(model="mixtral-8x7b-32768", temperature=0)
tools = [calculate_calories, get_nutritional_info]

# Get the prompt template from LangChain hub
prompt = hub.pull("hwchase17/react")

# Create the agent
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Test the agent
result = agent_executor.invoke({"input": "What are the calories and nutritional info for chicken breast?"})
print("Agent Response:", result["output"])

## Example 2: Agent with Multiple Function Tools

Agent that can call multiple tools to answer diet planning questions.

In [ ]:
@tool
def check_dietary_restrictions(restriction: str) -> str:
    """Check if a food item is suitable for common dietary restrictions."""
    restrictions_db = {
        "vegetarian": ["chicken", "salmon", "beef"],  # These are NOT vegetarian
        "gluten-free": ["bread", "pasta", "cereal"],  # These contain gluten
        "vegan": ["chicken", "salmon", "beef", "eggs", "milk"],  # Animal products
        "low-carb": ["rice", "bread", "pasta"]  # High carb foods
    }
    return f"Foods to avoid for {restriction}: {', '.join(restrictions_db.get(restriction.lower(), []))}"

@tool
def suggest_meal_plan(calories_target: int, diet_type: str) -> str:
    """Suggest a meal plan based on calorie target and diet type."""
    meal_suggestions = {
        "vegetarian": ["Grilled vegetables with paneer", "Lentil curry", "Vegetable stir-fry"],
        "high-protein": ["Grilled chicken breast", "Salmon fillet", "Egg whites"],
        "low-carb": ["Grilled vegetables", "Salads with olive oil", "Meat dishes"]
    }
    suggestions = meal_suggestions.get(diet_type.lower(), ["Balanced meal"])
    return f"For {calories_target} calories and {diet_type} diet, suggested meals: {', '.join(suggestions)}"

# Create an enhanced agent with more tools
llm = ChatGroq(model="mixtral-8x7b-32768", temperature=0)
tools_extended = [calculate_calories, get_nutritional_info, check_dietary_restrictions, suggest_meal_plan]

prompt = hub.pull("hwchase17/react")
agent_extended = create_react_agent(llm, tools_extended, prompt)
agent_executor_extended = AgentExecutor(agent=agent_extended, tools=tools_extended, verbose=True)

# Test with a more complex query
result = agent_executor_extended.invoke({
    "input": "I need a vegetarian meal plan for 2000 calories. What meals do you suggest and what should I avoid?"
})
print("\nAgent Response:", result["output"])

## Example 3: Agent with Memory (Conversation History)

Agent that remembers previous interactions in a conversation.

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentType, initialize_agent

# Create memory for conversation history
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Initialize agent with memory using the deprecated but still functional approach
llm_with_memory = ChatGroq(model="mixtral-8x7b-32768", temperature=0)

# You can also use a simpler approach with conversation buffer
conversation_history = []

def add_to_history(role: str, content: str):
    """Helper to add messages to conversation history."""
    conversation_history.append({"role": role, "content": content})

# Simulate multi-turn conversation
print("=== Multi-turn Conversation Example ===\n")

# Turn 1
user_input_1 = "I'm trying to gain muscle. What's a good daily calorie intake for muscle building?"
print(f"User: {user_input_1}")
add_to_history("user", user_input_1)

# Agent processes with context
response_1 = agent_executor_extended.invoke({"input": user_input_1})
print(f"Agent: {response_1['output']}\n")
add_to_history("assistant", response_1['output'])

# Turn 2 - Using conversation history context
user_input_2 = "What are the top 3 high-protein foods I should include?"
print(f"User: {user_input_2}")
add_to_history("user", user_input_2)

response_2 = agent_executor_extended.invoke({"input": user_input_2})
print(f"Agent: {response_2['output']}\n")
add_to_history("assistant", response_2['output'])

# Print conversation summary
print("\n=== Conversation History ===")
for msg in conversation_history:
    print(f"{msg['role'].upper()}: {msg['content'][:100]}...")

## Example 4: Custom Agent with Error Handling

Agent with built-in error handling and input validation.

In [ ]:
@tool
def validate_and_calculate_daily_intake(age: int, weight: float, activity_level: str) -> str:
    """Calculate daily caloric needs based on user parameters with validation."""
    try:
        if not (1 <= age <= 120):
            return "Error: Age must be between 1 and 120"
        if not (10 <= weight <= 500):
            return "Error: Weight must be between 10 and 500 kg"
        
        valid_activities = ["sedentary", "lightly_active", "moderately_active", "very_active"]
        if activity_level.lower() not in valid_activities:
            return f"Error: Activity level must be one of {valid_activities}"
        
        # Simplified calculation (Basal Metabolic Rate approximation)
        bmr = 10 * weight + 6.25 * age - 5
        activity_multipliers = {
            "sedentary": 1.2,
            "lightly_active": 1.375,
            "moderately_active": 1.55,
            "very_active": 1.725
        }
        
        tdee = bmr * activity_multipliers[activity_level.lower()]
        return f"Based on age {age}, weight {weight}kg, and {activity_level} activity: Daily caloric needs = {tdee:.0f} calories"
    
    except Exception as e:
        return f"Error in calculation: {str(e)}"

# Create robust agent with error handling
llm_robust = ChatGroq(model="mixtral-8x7b-32768", temperature=0)
tools_robust = [validate_and_calculate_daily_intake, get_nutritional_info, suggest_meal_plan]

prompt = hub.pull("hwchase17/react")
agent_robust = create_react_agent(llm_robust, tools_robust, prompt)
agent_executor_robust = AgentExecutor(
    agent=agent_robust, 
    tools=tools_robust, 
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)

# Test with valid input
print("=== Test with Valid Input ===")
result = agent_executor_robust.invoke({
    "input": "I'm 30 years old, weigh 75kg, and have a moderately active lifestyle. What's my daily caloric need?"
})
print(f"Result: {result['output']}\n")

# Test with invalid input
print("=== Test with Invalid Input ===")
try:
    result = agent_executor_robust.invoke({
        "input": "I'm 200 years old and weigh 5kg - what should I eat?"
    })
    print(f"Result: {result['output']}")
except Exception as e:
    print(f"Agent handled error gracefully: {str(e)}")

## Example 5: Agent Chain - Sequential Tool Execution

Agent that chains multiple tools together in sequence to solve complex problems.

In [ ]:
from langchain.agents import AgentType, initialize_agent

@tool
def compile_meal_prep_guide(meal_name: str, servings: int) -> str:
    """Create a meal prep guide with ingredients and instructions."""
    meal_guides = {
        "grilled chicken breast": {
            "ingredients": ["Chicken breast", "Olive oil", "Salt", "Pepper"],
            "instructions": "Season chicken, grill for 6-7 minutes per side at 180°C",
            "prep_time": "15 minutes"
        },
        "rice and beans": {
            "ingredients": ["Brown rice", "Black beans", "Onion", "Garlic"],
            "instructions": "Cook rice for 30 minutes, simmer beans for 45 minutes, combine",
            "prep_time": "50 minutes"
        },
        "vegetable stir-fry": {
            "ingredients": ["Mixed vegetables", "Soy sauce", "Garlic", "Ginger"],
            "instructions": "Heat wok, stir-fry vegetables for 5-7 minutes on high heat",
            "prep_time": "20 minutes"
        }
    }
    
    guide = meal_guides.get(meal_name.lower(), None)
    if guide:
        return f"Meal: {meal_name}\nServings: {servings}\nIngredients: {', '.join(guide['ingredients'])}\nInstructions: {guide['instructions']}\nPrep Time: {guide['prep_time']}"
    return f"Meal guide for '{meal_name}' not found in database"

# Create a specialized agent for meal planning
llm_meal_planner = ChatGroq(model="mixtral-8x7b-32768", temperature=0)
tools_meal_planner = [
    validate_and_calculate_daily_intake,
    get_nutritional_info,
    suggest_meal_plan,
    compile_meal_prep_guide
]

prompt = hub.pull("hwchase17/react")
agent_meal_planner = create_react_agent(llm_meal_planner, tools_meal_planner, prompt)
agent_executor_meal_planner = AgentExecutor(
    agent=agent_meal_planner, 
    tools=tools_meal_planner, 
    verbose=True,
    max_iterations=15
)

# Complex query that requires multiple tools
print("=== Complex Multi-Step Agent Query ===\n")
result = agent_executor_meal_planner.invoke({
    "input": "I need a high-protein meal plan. First calculate my daily caloric needs (I'm 25, weigh 70kg, very active), then suggest high-protein meals, and finally give me a prep guide for grilled chicken breast for 4 servings."
})
print(f"\nFinal Result: {result['output']}")

## Key Concepts Summary

**What are LangChain Agents?**
- Agents use an LLM to determine which tools to call and in what order
- They follow a reasoning loop: Think → Act → Observe → Repeat
- Excellent for complex multi-step tasks that require decision-making

**When to use Agents:**
- Multi-tool workflows where tool selection depends on user input
- Tasks requiring reasoning about tool sequencing
- Problems where the solution path isn't predetermined
- Interactive applications needing dynamic tool usage

**Common Agent Types:**
- **ReAct Agent**: Reason + Act pattern (recommended)
- **OpenAI Functions Agent**: For models with function-calling capabilities
- **Conversational Agent**: For multi-turn dialogue with memory

**Best Practices:**
1. Define clear tool descriptions - agents rely on these to decide tool usage
2. Include input validation in tools to prevent errors
3. Use `max_iterations` to prevent infinite loops
4. Enable `verbose=True` during development for debugging
5. Handle parsing errors gracefully
6. Keep tools focused and single-purpose
7. Provide example outputs in tool descriptions